In [25]:
# Import live code changes in
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os
import pandas as pd
from rasterstats import zonal_stats
import pandas as pd
import geopandas as gpd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Set filepaths and provide data info

In [26]:
root = Path.cwd().parent # find project root
basin_path = os.path.join(root, 'outputs', 'boundaries', 'analysis_basins.gpkg')
risk_map_path = os.path.join(root, 'outputs', 'flood', 'risk', 'maps')
output_path = os.path.join(root, 'outputs', 'flood', 'risk', 'basins')
raster_info_list = [
    {"RP": 10, "file": os.path.join(risk_map_path, 'JRC_10_pub_cap_damages.tif')},
    {"RP": 20, "file": os.path.join(risk_map_path, 'JRC_20_pub_cap_damages.tif')},
    {"RP": 50, "file": os.path.join(risk_map_path, 'JRC_50_pub_cap_damages.tif')},
    {"RP": 75, "file": os.path.join(risk_map_path, 'JRC_75_pub_cap_damages.tif')},
    {"RP": 100, "file": os.path.join(risk_map_path, 'JRC_100_pub_cap_damages.tif')},
    {"RP": 200, "file": os.path.join(risk_map_path, 'JRC_200_pub_cap_damages.tif')},
    {"RP": 500, "file": os.path.join(risk_map_path, 'JRC_500_pub_cap_damages.tif')}
]

In [27]:
# Load basins
basin_df = gpd.read_file(basin_path)
# Initializ an empty DataFrame to store zonal statistics
results_df = pd.DataFrame()

#### Run overlay analysis 

In [28]:
for raster_info in raster_info_list:
    # Calculate zonal statistics for each rasters
    zs = zonal_stats(basin_df, raster_info['file'], stats='sum', geojson_out=True)
    # Prepare a DataFrame from the zonal statistics
    temp_df = pd.DataFrame({
        "FID": [feat['id'] for feat in zs],
        "GID_1": [feat['properties']['flpr_gid_1'] for feat in zs],
        "NAME": [feat['properties']['NAME'] for feat in zs],
        "HB_L6": [feat['properties']['HYBAS_ID_06'] for feat in zs],
        "Pr_L": [feat['properties']['MerL_Riv'] for feat in zs],
        "damages": [feat['properties']['sum'] for feat in zs]
    })
    # Add raster information to the DataFrame
    temp_df["RP"] = raster_info['RP']
    
    # Remove NaNs from damage column
    temp_df['damages'] = temp_df['damages'].fillna(0)

    # Concatenate teh temporary DataFrame with the results DataFrame
    results_df = pd.concat([results_df, temp_df], ignore_index=True)

# Save results
if not os.path.exists(output_path):
    os.makedirs(output_path)
results_df.to_csv(os.path.join(output_path, 'risk_basins.csv'))